# Aprendizaje Multietiqueta de Patrones Geométricos en Objetos de Herencia Cultural
# CNN Multilabeling through AlexNet
## Seminario de Tesis II, Primavera 2022
### Master of Data Science. Universidad de Chile.
#### Prof. guía: Benjamín Bustos - Prof. coguía: Iván Sipirán
#### Autor: Matías Vergara

El objetivo de este notebook es realizar predicciones multilabel sobre patrones geométricos mediante AlexNet.

## Montando Google Drive

In [70]:
# Mounting google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    root_dir = '../content/gdrive/MyDrive'
except:
    root_dir = '../'

In [125]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import math
from torch.nn.utils.rnn import pack_padded_sequence

%matplotlib inline
from matplotlib import pyplot as plt

## Selección de dataset y experimento

In [72]:
DS_FLAGS = ['base']
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
CROP_TIMES = 2
RANDOM_TIMES = 2
ELASTIC_TIMES = 3

use_pos_weights = True
pos_weights_factor = 2.5
NUM_LABELS = 30
use_testval = True
BATCH_SIZE = 100

In [73]:
# This cells builds the data_flags variable, that will be used
# to map the requested data treatment to folders
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = ['crop', 'randaug', 'elastic']
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

patterns_path = os.path.join(root_dir, 'patterns', data_flags) 
labels_path = os.path.join(root_dir, 'labels', data_flags)

if not (os.path.isdir(patterns_path) and os.path.isdir(labels_path)):
    raise FileNotFoundError("No existen directorios de datos para el conjunto de flags seleccionado. Verifique que el dataset exista en {}".format(
        (os.path.isdir(patterns_path), os.path.isdir(labels_path))))
print("Patterns set encontrado en {}".format(patterns_path))
print("Labels set encontrado en {}".format(labels_path))

exp_name = f"{NUM_LABELS}L"
exp_name += "_testval" if use_testval else ""
weights_str = str(pos_weights_factor)
weights_str = weights_str.replace('.','_')
exp_name += f'_weighted_{weights_str}' if use_pos_weights else ''
print(f"Nombre del experimento: {exp_name}")

output_dir = os.path.join(root_dir, "outputs", "alexnet", data_flags, exp_name)
os.makedirs(output_dir, exist_ok = True)

model_dir = os.path.join(root_dir, "models", "alexnet", data_flags)
model_path = os.path.join(model_dir, exp_name + '.pth')

print(f"Los resultados se guardarán en: {output_dir}")
print(f"Los resultados se guardarán en: {output_dir}")

Patterns set encontrado en ../patterns\base
Labels set encontrado en ../labels\base
Nombre del experimento: 30L_testval_weighted_2_5
Los resultados se guardarán en: ../outputs\alexnet\base\30L_testval_weighted_2_5
Los resultados se guardarán en: ../outputs\alexnet\base\30L_testval_weighted_2_5


In [74]:
labels_train = pd.read_json(os.path.join(labels_path, 'augmented_train_df.json'), orient='index')
labels_val = pd.read_json(os.path.join(labels_path, 'val_df.json'), orient='index')
labels_test = pd.read_json(os.path.join(labels_path, 'test_df.json'), orient='index')

In [75]:
labels_test.loc[['84a']]['outline']

84a    1
Name: outline, dtype: int64

In [76]:
def filter_labels(labels_df, freq=25, number_labels = None):
  """Filters a label dataframe based on labels frequency (number of events)

    Parameters:
    labels_df (DataFrame): dataframe of labels 
    freq (int): threshold frequency. Labels with a lower value will be filtered. 

    Returns:
    DataFrame: filtered labels dataframe

  """
  top_labels = None

  if not number_labels:
    filtered_df = labels_df.loc[:, labels_df.sum(axis=0) > freq]
    top_labels = filtered_df.sum().sort_values(ascending=False)
    return top_labels, 0

  if number_labels:
      filtered_labels = labels_df.shape[1]
      pivot = 0
      while filtered_labels > number_labels:
        #print(filtered_labels, number_labels, pivot)
        filtered_df = labels_df.loc[:, labels_df.sum(axis=0) > pivot]
        top_labels = filtered_df.sum().sort_values(ascending=False)
        filtered_labels = filtered_df.shape[1]
        pivot += 1
      print("Aplicando threshold {} para trabajar con {} labels".format(pivot, len(top_labels.values)))
      return top_labels, pivot

def filter_dfs(df, top_labels_df):
  df = df[df.columns.intersection(top_labels_df.index)]
  return df

def make_positive_weights(labels, factor=1):                        
    total = labels.values.sum()
    weights = [0.] * len(labels)
    for i, label in enumerate(labels):
      weights[i] = total/(factor * labels[i])
    return weights

def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [77]:
labels = pd.read_json(os.path.join(labels_path, 'augmented_train_df.json'), orient='index')
top_labels = filter_labels(labels, number_labels = NUM_LABELS)

Aplicando threshold 22 para trabajar con 30 labels


In [78]:
top_labels[0].values.sum()

1915

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_labels = pd.read_json(os.path.join(labels_path, 'augmented_train_df.json'), orient='index')
top_labels, _ = filter_labels(train_labels, number_labels=NUM_LABELS)
NUM_LABELS = len(top_labels) # la cantidad final de etiquetas a trabajar

if use_pos_weights:
    pos_weights = make_positive_weights(top_labels, pos_weights_factor)
    pos_weights = torch.Tensor(pos_weights).float().to(device)
else:
    pos_weights = None

# images_dir=os.path.join(root_dir, 'patterns', data_flags, 'train'),
# labels_file=os.path.join(root_dir, 'labels', data_flags, 'augmented_train_df.json'),
class KunischDataset(torch.utils.data.Dataset):

  def __init__(self, images_dir, labels_file, transform, top_labels, extra_labels = None, extra_images_dir = None):
    """
    Args:
        text_file(string): path to text file
        root_dir(string): directory with all train images
    """
    self.labels_frame = filter_dfs(pd.read_json(labels_file, orient='index'), top_labels)
    self.num_labels = len(top_labels)
    self.images_dir = images_dir
    self.labels_file = labels_file
    self.transform = transform
    self.flags = data_flags
    self.top_labels = top_labels
    self.extra_images_dir = None 

    # para crear conjunto test-val 
    if extra_labels:
      extra_labels_frame = filter_dfs(pd.read_json(extra_labels, orient='index'), top_labels)
      self.labels_frame = pd.DataFrame.append(self.labels_frame, extra_labels_frame)
      self.extra_images_dir = extra_images_dir

  def __len__(self):
    return len(self.labels_frame)

  def __getitem__(self, idx):
    img_id = self.labels_frame.iloc[idx].name + '.png'
    img_name = None
    for chapter in os.listdir(self.images_dir):
      if img_id in os.listdir(os.path.join(self.images_dir, chapter)):
        img_name = os.path.join(self.images_dir, chapter, img_id)
        break
    # caso test-val
    if (self.extra_images_dir is not None) and (img_name is None):
      for chapter in os.listdir(self.extra_images_dir):
        if img_id in os.listdir(os.path.join(self.extra_images_dir, chapter)):
          img_name = os.path.join(self.extra_images_dir, chapter, img_id)
          break
    if img_name is None:
      raise Exception(f'No se encontró la imagen para {img_id}')
    image = Image.open(img_name)
    image = image.convert('RGB')
    image = self.transform(image)
    labels = self.labels_frame.iloc[idx].values
    labels = np.array(labels)
    labels = torch.from_numpy(labels.astype('int'))
    #print(img_id, img_name, self.labels_frame.iloc[idx], self.labels_frame.iloc[idx].values, labels)
    sample = {'image': image, 'labels': labels, 'paths': img_name}
    return sample


# Alexnet requires 227 x 227
# Training
kunischTrainSet = KunischDataset(images_dir=os.path.join(patterns_path, 'train'),
                                 labels_file=os.path.join(labels_path, 'augmented_train_df.json'),
                                 transform=transforms.Compose([transforms.Resize((227, 227)),
                                                               transforms.ToTensor(),
                                                               transforms.Normalize(
                                                                   mean=[0.485, 0.456, 0.406],
                                                                   std=[0.229, 0.224, 0.225])]),
                                 top_labels=top_labels)

kunischTrainLoader = torch.utils.data.DataLoader(kunischTrainSet, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# Validation
kunischValidationSet = KunischDataset(images_dir=os.path.join(patterns_path, 'val'),
                                      labels_file=os.path.join(labels_path, 'val_df.json'),
                                      transform=transforms.Compose([transforms.Resize((227, 227)),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(
                                                                        mean=[0.485, 0.456, 0.406],
                                                                        std=[0.229, 0.224, 0.225])]),
                                      top_labels=top_labels)

kunischValidationLoader = torch.utils.data.DataLoader(kunischValidationSet, batch_size=BATCH_SIZE, shuffle=True,
                                                      num_workers=0)

# Test
kunischTestSet = KunischDataset(images_dir=os.path.join(patterns_path, 'test'),
                                labels_file=os.path.join(labels_path, 'test_df.json'),
                                transform=transforms.Compose([transforms.Resize((227, 227)),
                                                              transforms.ToTensor(),
                                                              transforms.Normalize(
                                                                  mean=[0.485, 0.456, 0.406],
                                                                  std=[0.229, 0.224, 0.225])]),
                                top_labels=top_labels,
                                extra_labels=os.path.join(labels_path, 'val_df.json' if use_testval else None),
                                extra_images_dir=os.path.join(patterns_path, 'val' if use_testval else None))

kunischTestLoader = torch.utils.data.DataLoader(kunischTestSet, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)


# Define the function for training, validation, and test
def alex_train(epoch, num_epochs, train_losses, learning_rate, w):
  alex_net.train()
  train_loss = 0
  TN = 0
  TP = 0
  FP = 0
  FN = 0
  preds_total = np.empty((1, NUM_LABELS), dtype=int)
  labels_total = np.empty((1, NUM_LABELS), dtype=int)
    
  for i, sample_batched in enumerate(kunischTrainLoader, 1):
      inputs = sample_batched['image'].to(device)
      labels = sample_batched['labels'].to(device)

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = alex_net(inputs)
      loss = criterion(outputs.float(), labels.float())
      loss.backward()
      optimizer.step()

      train_loss += loss.item()
      pred = (torch.sigmoid(outputs).data > 0.5).int()
      # print(pred)
      labels = labels.int()
      # print(labels)
      pred = (torch.sigmoid(outputs).data > 0.5).int()
      labels = labels.int()
      preds_total = np.concatenate((preds_total, pred.cpu()), axis=0)
      labels_total = np.concatenate((labels_total, labels.cpu()), axis=0)
    
      TP += ((pred == 1) & (labels == 1)).float().sum()  # True Positive Count
      TN += ((pred == 0) & (labels == 0)).float().sum()  # True Negative Count
      FP += ((pred == 1) & (labels == 0)).float().sum()  # False Positive Count
      FN += ((pred == 0) & (labels == 1)).float().sum()  # False Negative Count
      #print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP, TN, FP, FN))

  TP = TP.cpu().numpy()
  TN = TN.cpu().numpy()
  FP = FP.cpu().numpy()
  FN = FN.cpu().numpy()

  accuracy = (TP + TN) / (TP + TN + FP + FN)
  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  f1_score = 2 * (precision * recall) / (precision + recall)
  train_loss = train_loss / len(kunischTrainLoader.dataset) * BATCH_SIZE
  hs = hamming_score(preds_total, labels_total)
  train_losses.append([epoch, learning_rate, w, train_loss, TP, TN, FP, FN, accuracy, precision, recall, f1_score])
  # print statistics
  print('Train Trial [{}/{}], LR: {}, W: {}, Avg Loss: {:.4f}, Accuracy: {:.4f}, F1 score: {:.4f}, HS: {:.4f}'
        .format(epoch, num_epochs, learning_rate, w, train_loss, accuracy, f1_score, hs))
  return hs

def alex_valid(epoch, num_epochs, valid_losses, learning_rate, w):
  # Have our model in evaluation mode
  alex_net.eval()
  # Set losses and Correct labels to zero
  valid_loss = 0
  TN = 0
  TP = 0
  FP = 0
  FN = 0
  preds_total = np.empty((1, NUM_LABELS), dtype=int)
  labels_total = np.empty((1, NUM_LABELS), dtype=int)
  with torch.no_grad():
      for i, sample_batched in enumerate(kunischValidationLoader, 1):
          inputs = sample_batched['image'].to(device)
          labels = sample_batched['labels'].to(device)
          outputs = alex_net(inputs)
          loss = criterion(outputs.float(), labels.float())
          valid_loss += loss.item()
          pred = (torch.sigmoid(outputs).data > 0.5).int()
          labels = labels.int()
          preds_total = np.concatenate((preds_total, pred.cpu()), axis=0)
          labels_total = np.concatenate((labels_total, labels.cpu()), axis=0)
        
          TP += ((pred == 1) & (labels == 1)).float().sum()  # True Positive Count
          TN += ((pred == 0) & (labels == 0)).float().sum()  # True Negative Count
          FP += ((pred == 1) & (labels == 0)).float().sum()  # False Positive Count
          FN += ((pred == 0) & (labels == 1)).float().sum()  # False Negative Count
          # print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP,TN,FP,FN) )

      TP = TP.cpu().numpy()
      TN = TN.cpu().numpy()
      FP = FP.cpu().numpy()
      FN = FN.cpu().numpy()
      accuracy = (TP + TN) / (TP + TN + FP + FN)
      precision = TP / (TP + FP)
      recall = TP / (TP + FN)
      f1_score = 2 * (precision * recall) / (precision + recall)
      hs = hamming_score(preds_total, labels_total)
      valid_loss = valid_loss / len(kunischValidationLoader.dataset) * BATCH_SIZE  # 1024 is the batch size
      valid_losses.append(
          [epoch, learning_rate, w, valid_loss, TP, TN, FP, FN, accuracy, precision, recall, f1_score])
      # print statistics
      print('Valid Trial [{}/{}], LR: {}, W: {}, Avg Loss: {:.4f}, Accuracy: {:.4f}, F1 score: {:.4f}, HS: {:.4f}'
            .format(epoch, num_epochs, learning_rate, w, valid_loss, accuracy, f1_score, hs))
      return hs

def alex_test(epoch, num_epochs, pred_array, test_losses, learning_rate, w, show_images=1):
  # Have our model in evaluation mode
  alex_net.eval()
  # Set losses and Correct labels to zero
  test_loss = 0
  TN = 0
  TP = 0
  FP = 0
  FN = 0
  preds_total = np.empty((1, NUM_LABELS), dtype=int)
  labels_total = np.empty((1, NUM_LABELS), dtype=int)
    
  with torch.no_grad():
      for i,sample_batched in enumerate(kunischTestLoader, 1):
          print("CURRENT BATCH SIZE: ", BATCH_SIZE)
          inputs = sample_batched['image'].to(device)
          labels = sample_batched['labels'].to(device)
          paths = sample_batched['paths']
          outputs = alex_net(inputs)
          
          loss = criterion(outputs.float(), labels.float())
          test_loss += loss.item()
          pred = (torch.sigmoid(outputs).data > 0.5).int()
          # print(pred)
          labels = labels.int()
          # print(labels)
          pred_array.append([paths, test_loss, labels, pred])
          pred = (torch.sigmoid(outputs).data > 0.5).int()
          labels = labels.int()
          preds_total = np.concatenate((preds_total, pred.cpu()), axis=0)
          labels_total = np.concatenate((labels_total, labels.cpu()), axis=0)
          
          for j in range(0, min(BATCH_SIZE, show_images)): # j itera sobre ejemplos
              print(f"Mostrando imagen {j} del batch {i}")
              img = np.transpose(sample_batched['image'][j]) # imagen j 
              plt.imshow(img, interpolation='nearest')
              plt.show()
              labels_correctos = ""
              labels_predichos = ""
              for k in range(0, len(pred[j])):
                labels_correctos += (kunischTestSet.labels_frame.columns.values[k]+' ') if labels[j].cpu().detach()[k] else ""
                labels_predichos += (kunischTestSet.labels_frame.columns.values[k]+' ') if pred[j].cpu().detach()[k] else ""
              print("Labels correctos:")
              #print(labels[j].cpu().detach().numpy())
              print(labels_correctos)
              print("Labels predichos:")
              #print(pred[j].cpu().detach().numpy())
              print(labels_predichos)
              print("\n")
            
          TP += ((pred == 1) & (labels == 1)).float().sum()  # True Positive Count
          TN += ((pred == 0) & (labels == 0)).float().sum()  # True Negative Count
          FP += ((pred == 1) & (labels == 0)).float().sum()  # False Positive Count
          FN += ((pred == 0) & (labels == 1)).float().sum()  # False Negative Count
          # print('TP: {}\t TN: {}\t FP: {}\t FN: {}\n'.format(TP,TN,FP,FN) )

      TP = TP.cpu().numpy()
      TN = TN.cpu().numpy()
      FP = FP.cpu().numpy()
      FN = FN.cpu().numpy()
      accuracy = (TP + TN) / (TP + TN + FP + FN)
      precision = TP / (TP + FP)
      recall = TP / (TP + FN)
      f1_score = 2 * (precision * recall) / (precision + recall)
      hs = hamming_score(preds_total, labels_total)
      test_loss = test_loss / len(kunischTestLoader.dataset) * 1024  # 1024 is the batch size
      test_losses.append([epoch, learning_rate, w, test_loss, TP, TN, FP, FN, accuracy, precision, recall, f1_score])
      # print statistics
      print('Test Trial [{}/{}], LR: {}, W: {}, Avg Loss: {:.4f}, Accuracy: {:.4f}, F1 score: {:.4f}, HS: {:.4f}'
            .format(epoch, num_epochs, learning_rate, w, test_loss, accuracy, f1_score, hs))
      return hs

In [ ]:
def filter_labels(labels_df, freq=25, number_labels = None):
  """Filters a label dataframe based on labels frequency (number of events)

    Parameters:
    labels_df (DataFrame): dataframe of labels 
    freq (int): threshold frequency. Labels with a lower value will be filtered. 

    Returns:
    DataFrame: filtered labels dataframe

  """
  top_labels = None

  if not number_labels:
    filtered_df = labels_df.loc[:, labels_df.sum(axis=0) > freq]
    top_labels = filtered_df.sum().sort_values(ascending=False)
    return top_labels, 0

  if number_labels:
      filtered_labels = labels_df.shape[1]
      pivot = 0
      while filtered_labels > number_labels:
        #print(filtered_labels, number_labels, pivot)
        filtered_df = labels_df.loc[:, labels_df.sum(axis=0) > pivot]
        top_labels = filtered_df.sum().sort_values(ascending=False)
        filtered_labels = filtered_df.shape[1]
        pivot += 1
      print("Aplicando threshold {} para trabajar con {} labels".format(pivot, len(top_labels.values)))
      return top_labels, pivot

def filter_dfs(df, top_labels_df):
  df = df[df.columns.intersection(top_labels_df.index)]
  return df

def make_positive_weights(labels, factor=1):                        
    total = labels.values.sum()
    weights = [0.] * len(labels)
    for i, label in enumerate(labels):
      weights[i] = total/(factor * labels[i])
    return weights


In [ ]:
# Hyper Parameter Tuning
alex_net = models.alexnet(pretrained=True)
for param in alex_net.parameters():
    param.requires_grad = False
alex_net.classifier._modules['6'] = nn.Linear(4096, NUM_LABELS)

train_losses = []
validation_losses = []
num_epochs = 10

for epoch in range(num_epochs):
  learning_rate = round(np.exp(random.uniform(np.log(.0001), np.log(.01))), 4)  # pull geometrically
  w = round(np.exp(random.uniform(np.log(3.1e-7), np.log(3.1e-5))), 10)  # pull geometrically

  # Reset Model per test
  alex_net = models.alexnet(pretrained=True)
  alex_net.classifier._modules['6'] = nn.Linear(4096, NUM_LABELS)
  alex_net.to(device)

  optimizer = torch.optim.Adam(alex_net.parameters(), lr=learning_rate, weight_decay=w)
  criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
  
  train_losses_df = pd.DataFrame(train_losses)
  train_losses_df.to_csv(os.path.join(output_dir, 'loss_hypertrain.csv'))

  alex_valid(epoch, num_epochs, validation_losses, learning_rate, w)
  validation_losses_df = pd.DataFrame(validation_losses)
  validation_losses_df.to_csv(os.path.join(output_dir, 'loss_hyperval.csv'))

In [ ]:
# Training
from torch.optim.lr_scheduler import StepLR
train_losses = []
num_epochs = 40
learning_rate = 0.0001
w = 0.001

# Early Stopping
patience = 10
bad_epochs = 0
best_score = 0.0
best_weights = None

alex_net = models.alexnet(pretrained=True)
alex_net.classifier._modules['6'] = nn.Linear(4096, NUM_LABELS)
alex_net.to(device)

optimizer = torch.optim.Adam(alex_net.parameters(), lr=learning_rate, weight_decay=w)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

for epoch in range(num_epochs):
  score_train = alex_train(epoch, num_epochs, train_losses, learning_rate, w)
  score_valid = alex_valid(epoch, num_epochs, validation_losses, learning_rate, w)
  scheduler.step()
  print("")
  
  # Early Stopping
  if score_valid > best_score:
    bad_epochs = 0
    best_epoch = epoch
    best_score = score_valid
    best_weights = alex_net.state_dict()
  else:
    bad_epochs += 1
  
  if bad_epochs == patience:
    print("Out of patience!")
    print(f"Best epoch: {best_epoch}")
    break

torch.save(best_weights, model_path)

In [ ]:
# Testing
test_losses = []
test_pred = []
learning_rate = 0.0001
w = 0.001

# Reset Model
alex_net = models.alexnet(pretrained=True)
alex_net.classifier._modules['6'] = nn.Linear(4096, NUM_LABELS)
alex_net.load_state_dict(torch.load(model_path))
alex_net.to(device)

optimizer = torch.optim.Adam(alex_net.parameters(), lr=learning_rate, weight_decay=w)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
alex_test(1, 1, test_pred, test_losses, learning_rate, w, show_images = 1)
test_pred_df = pd.DataFrame(test_pred)

In [102]:
preds = test_pred[0][3].cpu().detach().numpy()
for i in range(1, len(test_pred)):
  pbi = test_pred[i][3].cpu().detach().numpy()
  preds = np.concatenate((preds, pbi), axis=0)
save_df = pd.DataFrame(preds)
save_df.to_csv(os.path.join(output_dir, 'predictions.csv'))

tops, _ = filter_labels(train_labels, number_labels = NUM_LABELS)
testval = pd.concat([labels_test, labels_val])
testval = filter_dfs(testval, tops)

print(preds.shape)
print(testval.shape)
print(f"HS final: {hamming_score(preds, testval.values)}")

Aplicando threshold 22 para trabajar con 30 labels
(100, 30)
(234, 30)
HS final: 0.4994444444444444
